In [10]:
## question similarity using word2vec(nepali model) and word mover distance

### Notebook Imports

In [1]:
from gensim.models import KeyedVectors

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import pandas as pd
import numpy as np

### load model


In [3]:
model = KeyedVectors.load_word2vec_format('nepali_embeddings_word2vec.txt', binary=False)

In [11]:
doc = pd.read_csv('dataset/modified.txt',sep=',')
doc.head()


Sentence Final Intent
0              तिम्रो नाम के हो          परि
1  तिमिलाई के भनेर चिन्छन अरुले          परि
2     तिमिलाई बोलाउनि नाम के हो          परि
3       म तिमिलाई के भनेर बोलाउ          परि
4        तिमिलाई के भनेर चिन्छन          परि

In [12]:
intent_count=pd.DataFrame(doc['Final Intent'].value_counts())
intent_count['intent']=intent_count.index

In [13]:
intent_count.index = np.arange(len(intent_count))

intent_count.columns = ['Count','Intent']

In [14]:
intent_count = intent_count[['Intent','Count']]
intent_count.head()


# intent_count.shape

Intent  Count
0           कोक    102
1    चिकेन ससेज     90
2  चिकेन लेगपीस     90
3     बफ सुकुटी     90
4   चिकेन ललिपप     90

#### saving the dataframe with count for further processing

In [15]:
# intent_count.to_csv('follow_up_qsn_v1_1_new.csv',index=None)

In [9]:
# checking word mover distance
model.wmdistance('म राम्रो छु','म ठिक छु')

0.8173072042066036

In [10]:
def most_similar(sentence, corpus,model):
    ''' 
    sentence: question to compare
    corpus: dataset containing all questions
    model: word2vec pretrained  model on nepali corpus
    returns most closest intent to the given sentence and its corresponding similarity measure 
    '''
    similar=[]
    sent_list = []
    for sent in corpus.values:
        sim = model.wmdistance(sentence,sent[0])
        similar.append(sim)
    most_sim_list= sorted(range(len(similar)), key=lambda x: similar[x])[:1]
    max_sim_value = max(similar)
    for i in most_sim_list:
        sent_list.append(corpus.iloc[i,1])
    return sent_list[0],max_sim_value

In [40]:
sent1='ठिक म'
intent,max_sim=most_similar(sent1,doc,model)

In [41]:
intent

'मोमो'

In [42]:
max_sim

1.261068847954568

### reading the dataset with respones

In [18]:
df_with_res = pd.read_csv('dataset/follow_up_qsn_v1_1_new.csv',sep=',')

ParserError: Error tokenizing data. C error: Expected 3 fields in line 22, saw 4


In [44]:
df_with_res.tail()

Intent  Count                                           response
44       फेन्टा      8  यहाँ फेन्टा पाइन्छ। हामी सँग जम्बो फेन्टा र सा...
45     स्प्राइट      8  यहाँ स्प्राइट पाइन्छ। हामी सँग जम्बो स्प्राइट ...
46  रेस्टुरेन्ट      7  यो रेस्टुरेन्टको नाम ग्रीन भ्याली हो र हजुर ला...
47       कम्पनि      7  मलाई पाइला टेक्नोलोजीले बनाएको हो | पाइला टेक्...
48         अन्य      0  सरि तपाइको प्रश्न यो रेस्टुरेन्ट सम्बन्धि हुनु...

In [45]:
def respond(df_with_res,intent,max_sim):
    '''df_with_res: dataframe that has response
        max_idx: intent_returned by the similarity function
        max_sim: measure of how similar the returned setence is with the given sentence
    '''
    if max_sim >0.7:
        res=df_with_res.loc[df_with_res['Intent'] == intent]['response']
    else:
        res= df_with_res.loc[df_with_res['Intent'] == 'अन्य']['response']
    return res.values[0]

In [46]:
respond(df_with_res=df_with_res, intent=intent, max_sim=max_sim)

'यहाँ मोमो पाइन्छ र हामी सँग चिकेन मोमो बफ मोमो भेज मोमो र सी मोमो  छ| मोमो बन्न पन्ध्र मिनेट लाग्छ|'

In [54]:
tgt = doc['Final Intent']

In [48]:
# t2 = doc['Sentence']

In [58]:
t2 = doc.copy()

In [49]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(test_size=0.1, random_state=42, n_splits=1)

In [59]:
for train_index, test_index in sss.split(t2,tgt):
    x_train, x_test = t2[t2.index.isin(train_index)], t2[t2.index.isin(test_index)]
    y_train, y_test = t2.loc[t2.index.isin(train_index),"Final Intent"], t2.loc[t2.index.isin(test_index),"Final Intent"]

    
    
   

[618  64 530 ... 263 159  76] [ 868  666 1236 1129  878  497 1467  415 1140  370  564  334   32  255
  178 1097 1124  957 1373  150  952 1113  115 1391 1451 1365 1340  960
 1461 1442 1260  297  809 1332 1319  287  778   97  602  900 1030  891
  542 1427   25  226   56  189  306  341 1169  218  702  167  907  755
  284 1481 1234  734  369  120  553 1505  547 1428 1117  700  265   73
  326  173  235   10  465 1042   78  354 1285 1399  576  970 1203  744
 1041 1159 1499 1496  689  609  494 1078  532 1171 1170 1010 1277  420
  612 1004  522  471  166  651 1069  387  648  789  740 1018 1410  487
 1334  384  448  845  820 1143  643  601  813 1190  888   50 1230  127
 1267  946  244  293 1316 1275 1444  406  405   92  211  979 1356  774
  403 1375  675  500  107 1259  258  458   60  771   20]


In [60]:
x_train.shape

(1358, 2)

In [61]:
y_train.shape

(1358,)

In [62]:
x_test.shape

(151, 2)

In [63]:
y_test.shape

(151,)

In [159]:
x_test

Sentence Final Intent
10               तिम्रो काम के हो          परि
20               के हो तिम्रो नाम          परि
25                      काम के हो          परि
32      तिम्रो कम्पनीको बारेमा भन       कम्पनि
50    नाम के हो यो रेस्टुरेन्ट को  रेस्टुरेन्ट
...                           ...          ...
1467       खुकुरी रम सक्केको हो र       खुकुरी
1481   खुकुरी रम कति रुपैयामा आउछ       खुकुरी
1496    कति पैसा पर्नी हो वाईनलाइ         वाईन
1499       कति रुपैयामा हो वाईनको         वाईन
1505    कति पैसा पर्नी हो वाईनलाइ         वाईन

[151 rows x 2 columns]

In [171]:
for sent in x_test.loc[:,'Sentence']:
    


ValueError: too many values to unpack (expected 2)

In [184]:
for x in x_test['Sentence'].values:
    print(x)

तिम्रो काम के हो
के हो तिम्रो नाम
काम के हो
तिम्रो कम्पनीको बारेमा भन
नाम के हो यो रेस्टुरेन्ट को
परि आज स्पेशल के हो
आज हामीले यहाको स्पेशल अर्डर गरे भने के आउछ
आज  डिसकाउन्ट छ
अफर छ
मलाई वाईफाईको पासवोर्ड चाहियो
वाईफाईको इस्पिड कस्तो छ
पिजा छ कि छैन 
कुन कुन पिजा छ
पिजालाई कति कुर्नु पर्छ
पिजा पाउछ कि पाउदैन
चिज पिजाको पैसा कति हो
कति पर्नी हो चिज पिजालाइ
चिज पिजाको पैसा कति पर्छ
मसरूम वाला पिजा पाइन्छ
मसरूम पिजाको पैसा कति हो
मसरूम पिजा बनाउन मिल्छ कि मिल्दैन
चिक्कन पिजा छ कि नाइँ
चिक्कन पिजा पाउला
चिक्कन पिजा कति रुपैयामा आउछ
मिक्स पिजालाई कति तिर्नु पर्नी हो
मिक्स पिजा बनाउन मिल्छ कि नाइँ
कति रुपैयामा हो मिक्स पिजाको
मोमो बन्छ कि नाई
कस्तो कस्तो मोमो छ
मोमोमा कुन छ
कति समय लाग्छ मोमो बन्न
भेज मोमो छ
भेज मोमो पाउला कि नाई
भेज वाला मोमो पाउछ कि पाउदैन
चिकेन मोमोलाई कति हो
चिकेन वाला मोमो पाउछ कि पाउदैन
चिकेन मोमो कतिमा पाउछ होला
बफ मोमोलाई कति हो
बफ मोमो कतिमा पाउछ होला
बफ मोमोलाइ कति पर्न जान्छ
सी मोमो कतिमा पाइन्छ
सी मोमो छ कि नाइँ
कति रुपैयामा हो सी मोमोको
चाउमिन पाउछ
चाउमिन बन्छ

In [191]:
def most_similar(dataset, corpus,model):
    ''' 
    sentence: question to compare
    corpus: dataset containing all questions
    model: word2vec pretrained  model on nepali corpus
    returns most closest intent to the given sentence and its corresponding similarity measure 
    '''
    
    
    similar=[]
    sent_list = []
    for s in dataset.values:
        for sent in corpus.values:
            sim = model.wmdistance(sent,s)
            similar.append(sim)
    
    return similar

#         most_sim_list= sorted(range(len(similar)), key=lambda x: similar[x])[:1]
        
# #         max_sim_value = max(similar)
#         for i in most_sim_list:
#             sent_list.append(corpus.iloc[i,1])
#     return sent_list

In [192]:
most_similar(x_test['Sentence'],doc,model)

AttributeError: 'Sequential' object has no attribute 'wmdistance'

### database connectivity

In [32]:
# import os
# import pymysql

In [63]:
# host = os.getenv('MYSQL_HOST')
# port = os.getenv('MYSQL_PORT')
# user = os.getenv('MYSQL_USER')
# password = os.getenv('MYSQL_PASSWORD')
# database = os.getenv('MYSQL_DATABASE')

# conn = pymysql.connect(
#     host=host,
#     port=int(3306),
#     user="root",
#     passwd=password,
#     db="NLP",
#     charset='utf8mb4')

In [64]:
# conn.close()

In [20]:
# df = pd.read_sql_query("SELECT response FROM follow_up_qsn_v2 WHERE Intent='{}'".format(max_idx[0]),
#     conn)
# df.iloc[0,0]

NameError: name 'max_idx' is not defined